In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

import os
from tqdm import tqdm
import json
import signal
import argparse
import wandb
import pandas as pd

import sys
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,confusion_matrix
from pathlib import Path

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to 

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
sys.argv = [
    'notebook',  
    '--modelname' ,  'Qwen/Qwen2-VL-72B-Instruct' , 
    '--data', '/root/home/data',
    '--data_path_files','/mnt/Gbenga_Enemy/ramy/WACV-2025-Workshop-ViGIR',
    '--results_dir', '/mnt/Gbenga_Enemy/ramy/results',
]

In [4]:
parser = argparse.ArgumentParser(description="A script to run V-LLMs on different image classification datasets")

In [5]:
parser.add_argument("--modelname", type=str, required=True, help="The name of the V-LLM model")
parser.add_argument("--data", type=str, required=True, help="Path to the data")
parser.add_argument("--data_path_files", type=str, required=True, help="Path to the image data dir")
parser.add_argument("--results_dir", type=str, required=True, help="Folder name to save results")

args = parser.parse_args()

In [6]:
valid_file=os.path.join(args.data_path_files,'Validation_Set.json')
test_file=os.path.join(args.data_path_files,'Testing_Set.json')

with open(valid_file, 'r') as file:
    valid = json.load(file) 

with open(test_file, 'r') as file:
    test = json.load(file) 

In [7]:
def display_image(image_path):
    img = mpimg.imread(image_path)
    plt.figure(figsize=(20, 20))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [8]:
directory_path=Path(args.data)

file_dict = {}


for file in directory_path.iterdir():
    if file.is_file() and 'collage' in file.name:
        parts = file.stem.split('_')  
        number = parts[-1]
        if number.isdigit():
            file_dict[int(number)] = str(file)

In [9]:
baseline_question = ["Given this collage of six images of the exact same area, collected over a period of 10 years. Are there any ephemeral gully appearances? Answer by Yes or No and nothing else."]

In [10]:
def prepare_inference_inputs(path, baseline_question):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": path},
                {"type": "text", "text": baseline_question},
            ],
        }
    ]
    

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(     text=[text],      images=image_inputs,      padding=True,    return_tensors="pt",)
    
    return inputs


In [19]:
# inputs=prepare_inference_inputs("https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", "Describe this image.")
# inputs = inputs.to("cuda")

In [11]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-72B-Instruct")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    args.modelname,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
).eval()

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 38/38 [07:53<00:00, 12.45s/it]


In [12]:
prompts = [
    "Given these six images of the exact same area and collected over a period of 10 years, do you see a low point in the terrain? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, do narrow, winding paths or channels appear? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there winding paths that become intermittent recurrent? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there any linear depressions or ruts which appear more pronounced along natural drainage lines or slopes? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there narrow and shallow channels which appear intermittently deeper or more indented into the soil? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there areas where soil appears disturbed or vegetation is removed? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, does a specific path lack vegetation, suggesting an evolving or emerging channel? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there varying types and levels of coarseness in the texture of the soil? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there clear starting and ending points of potential channels? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there small rills or grooves indicating water flow? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, is there a varying exposure of lighter or darker colored soil? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there sediment accumulations forming? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there signs of water activity, like soil clumps or crusting, that appear or intensify in specific areas? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there any branching patterns that resemble temporary streams? Answer with yes or no only!",
    "Given these six images of the exact same area and collected over a period of 10 years, are there indications of nearby human activity, such as tillage or machinery tracks? Answer with yes or no only!"
]

In [15]:
count=0
saving_response={}

for key,info in tqdm(test.items()):
    path=file_dict[int(key)]
    print(key,path)
    count+=1
    saving_response[key]=[]
    for question in prompts:
        
        inputs=prepare_inference_inputs(path, question)
        inputs = inputs.to("cuda")
    
        generated_ids = model.generate(**inputs, max_new_tokens=128)
        generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
        response = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        saving_response[key].append([info,question,response])

    
with open(os.path.join(args.results_dir,f'test_{args.modelname[-21:]}-QAs.json'), "w") as file:
    json.dump(saving_response, file)
    
    

  0%|                                                                                           | 0/311 [00:00<?, ?it/s]

415 /root/home/data/pos_collage_415.jpg


  0%|▎                                                                                  | 1/311 [00:07<36:33,  7.08s/it]

1020 /root/home/data/neg_collage_1020.jpg


  1%|▌                                                                                  | 2/311 [00:14<36:17,  7.05s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████▍| 309/311 [36:06<00:14,  7.01s/it]

131 /root/home/data/neg_collage_131.jpg


100%|████████████████████████████████████████████████████████████████████████████████▋| 310/311 [36:13<00:07,  7.01s/it]

1133 /root/home/data/neg_collage_1133.jpg


100%|█████████████████████████████████████████████████████████████████████████████████| 311/311 [36:20<00:00,  7.01s/it]
